# Get Started Task-2 Clue-2 

## Initiate the boto3 session and SageMaker session
- import libraries
- initiate sagemaker client, and s3 resources
- fetch sagemaker default bucket

In [ ]:
# import sagemaker and s3 libraries
import sagemaker
import boto3

In [ ]:
# get the region and sagemaker session
region = boto3.Session().region_name
session = sagemaker.Session()

# initiate sagemaker client and s3 resources
sm = boto3.Session().client(service_name='sagemaker', region_name=region)
s3 = boto3.resource('s3')

In [ ]:
all_buckets = s3.buckets.all()
for bucket in all_buckets:
    if (bucket.name.startswith('sagemaker')):
        default_bucket = bucket.name
        print ("SageMaker Default Bucket for Training Data: ", default_bucket)

In [ ]:
# create train_data_s3 as train data s3 path
BUCKET = default_bucket
TRAIN_DATA_PREFIX = 'radiography_train_data'
TEST_DATA_PREFIX = 'radiography_test_data'
train_data_s3 = 's3://{}/{}'.format(BUCKET, TRAIN_DATA_PREFIX)
train_data_s3

## Update the training script
- download the transfer_learning_resnet.py from inventory
- update the classes number in the transfer_learning_resnet.py file
    - the solution can be found in Task2, clue 1
- upload the transfer_learning_resnet.py to the default bucket, s3://{BUCKET}/{training_jobs}

In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
from datetime import datetime
role = get_execution_role()

In [ ]:
src_code_s3 = 's3://{}/{}'.format(BUCKET, 'training_jobs')
training_job_output_s3 = 's3://{}/{}'.format(BUCKET, 'training_jobs_output')

## Initiate Model Estimator

In [ ]:
# Encapsulate training on SageMaker with PyTorch:
# The complete initiation of the estimator can be found in Task2, clue 2
train_estimator = PyTorch(entry_point='transfer_learning_resnet.py',
                          role=role,
                          framework_version='1.8.1',
                          py_version='py3',
                          debugger_hook_config=False,
                          instance_count=1,
                          instance_type='ml.p2.xlarge',
                          output_path=training_job_output_s3,
                          code_location=src_code_s3,
                          hyperparameters={'epochs': 2,
                                           'lr': 0.001,
                                           'batch-size': 8
                          }
)

In [ ]:
# Setting up File-system to import data from S3
data_channels = {'train': sagemaker.inputs.TrainingInput(
                          s3_data_type='S3Prefix',
                          s3_data=train_data_s3,
                          content_type='image/jpeg',
                          input_mode='File'),
                 
                 'val': sagemaker.inputs.TrainingInput(
                        s3_data_type='S3Prefix',
                        s3_data=train_data_s3,
                        content_type='image/jpeg',
                        input_mode='File')
}

In [ ]:
# Launching SageMaker training job
# start the training job with estimator
# The complete initiate of the training job can be found in Task2, clue 3
# {fill the code here} 